# 프롬프트 엔지니어링과 Tool Use를 활용한 미국 주식 분석 서비스 (Advanced)
이 코드는 LLM의 활용도를 극대화하기 위해 여러 기능을 추가했습니다. 주요 개선 사항으로는 1) 전일 대비 변동폭과 거래량 등 기술적 지표의 분석 및 해석, 2) 뉴스 헤드라인을 기반으로 한 시장 감성 분석, 3) 시스템 및 사용자 프롬프트의 최적화가 있습니다. 이러한 개선을 통해 코드는 단순한 데이터 조회를 넘어서, Claude의 분석 능력을 최대한 활용하여 투자자들에게 더욱 가치 있고 통찰력 있는 정보를 제공합니다.
  

## 1. 환경 설정

In [1]:
!python -V

Python 3.10.14


In [2]:
# 사용하기 전에 필요한 설치:
%pip install pandas numpy requests lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
%load_ext autoreload
%autoreload 2

import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = "../.."
add_python_path(module_path)

python path: /home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr is added
sys.path:  ['/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/13_tool_use_stock_analysis', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr']


Setup AWS Bedrock and Claude Integration

In [4]:
from pprint import pprint
from termcolor import colored
from utils import bedrock
from utils.bedrock import *
from anthropic import AnthropicBedrock
from langchain_aws import ChatBedrockConverse

# Initialize Bedrock client
client = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

# # Initialize Claude model
# claude = ChatBedrockConverse(
#     model="anthropic.claude-3-sonnet-20240229-v1:0",
#     temperature=0,
#     max_tokens=4096
# )

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-5-Sonnet': 'anthropic.claude-3-5-sonnet-20240620-v1:0',
 'Claude-V3-5-V-2-Sonnet': 'anthropic.claude-3-5-sonnet-20241022-v2:0',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Opus': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.l

## 2. Tool 정의

Alpha Vantage를 사용해 최신 주식 데이터 가져오기
- [사전 사항] Alpha Vantage API 발급 받기 https://www.alphavantage.co/support/#api-key
- [참고] Alpha Vantage의 문서에 따르면, 프리티어에서 기본적으로 데이터는 "각 거래일 종료 시점에 업데이트" 됨

In [5]:
import requests
api_key = "XXXX"  # Alpha Vantage API 키

# 최신 주식 데이터를 가져오기 위한 함수
def get_stock_data(ticker):
    """Alpha Vantage에서 주식 데이터를 가져오는 함수"""
    try:
        url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={ticker}&apikey={api_key}"
        
        response = requests.get(url)
        data = response.json()
        
        if "Global Quote" not in data:
            return {
                'status': 'error',
                'message': 'No data available'
            }
            
        quote = data["Global Quote"]
        
        return {
            'status': 'success',
            'data': {
                'date': quote["07. latest trading day"],
                'open': float(quote["02. open"]),
                'high': float(quote["03. high"]),
                'low': float(quote["04. low"]),
                'volume': int(quote["06. volume"])
            }
        }
    except Exception as e:
        return {
            'status': 'error',
            'message': str(e)
        }


# 기술적 지표 데이터를 가져오기 위한 함수
def analyze_technical_indicators(data):
    """기술적 지표 계산"""
    try:
        # 전일 대비 변동폭
        daily_range = data['high'] - data['low']
        # 거래량 분석
        volume_analysis = 'high' if data['volume'] > 1000000 else 'moderate' if data['volume'] > 500000 else 'low'
        
        return {
            'daily_range': daily_range,
            'volume_analysis': volume_analysis
        }
    except Exception as e:
        return {'error': str(e)}


# 시장 감성 분석을 위한 함수
def get_market_sentiment(ticker):
    """뉴스 헤드라인 기반 시장 감성 분석"""
    try:
        # Alpha Vantage News API 호출
        url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={api_key}"
        response = requests.get(url)
        data = response.json()
        
        return {
            'sentiment_score': data.get('sentiment_score', 0),
            'recent_news': data.get('feed', [])[:3]
        }
        
    except Exception as e:
        return {'error': str(e)}
        


## 3. Claude 호출

(참고) Tool Use 패턴의 일반적인 흐름
1. 도구 사용 필요성 판단 및 요청
2. 도구 실행 결과를 바탕으로 최종 응답 생성


In [8]:
def process_stock_request(client, ticker):
    """Bedrock Converse API를 사용하여 주식 데이터 처리"""
    try:
        # Tool 설정
        tool_config = {
            'tools': [{
                'toolSpec': {
                    # Tool의 이름, 설명, 입력 파라미터 정의
                    'name': 'get_stock_data',
                    'description': 'Fetches current stock data from Alpha Vantage API',
                    'inputSchema': {
                        'json': {
                            'type': 'object',
                            'properties': {
                                'ticker': {
                                    'type': 'string',
                                    'description': 'Stock ticker symbol'
                                }
                            },
                            'required': ['ticker']
                        }
                    }
                }
            },
            {
                'toolSpec': {
                    'name': 'analyze_technical_indicators',
                    'description': 'Calculates technical indicators from stock data',
                    'inputSchema': {
                        'json': {
                            'type': 'object',
                            'properties': {
                                'data': {
                                    'type': 'object',
                                    'properties': {
                                        'high': {'type': 'number'},
                                        'low': {'type': 'number'},
                                        'volume': {'type': 'number'}
                                    },
                                    'required': ['high', 'low', 'volume']
                                }
                            },
                            'required': ['data']
                        }
                    }
                }
            },
            {
                'toolSpec': {
                    'name': 'get_market_sentiment',
                    'description': 'Fetches and analyzes market sentiment from news',
                    'inputSchema': {
                        'json': {
                            'type': 'object',
                            'properties': {
                                'ticker': {
                                    'type': 'string',
                                    'description': 'Stock ticker symbol'
                                }
                            },
                            'required': ['ticker']
                        }
                    }
                }
            }],
            'toolChoice': {'auto': {}}  # LLM이 필요한 tool을 자유롭게 선택할 수 있도록 설정
        }

        # System prompt
        # 사용된 프롬프트 엔지니어링 기법: 역할 부여 (advanced financial analyst), 카테고리와 숫자를 활용한 계층 구조로 명확하고 직접적인 지시하기, 아웃풋 포맷팅 규칙 지정
        system_prompt = [{
            "text": """You are an advanced financial analyst. Analyze the stock data and provide:
            
            1. Technical Analysis:
            - Interpret the daily price range and volume patterns
            - Identify potential support/resistance levels
            - Highlight any significant price movements
            
            2. Market Sentiment:
            - Analyze recent news sentiment
            - Provide context for market reactions
            - Highlight key news impacts
            
            3. Trading Insights:
            - Suggest potential entry/exit points
            - Identify risk factors
            - Provide comparative analysis with sector performance
            
            4. Investment Recommendations:
            - Short-term trading opportunities
            - Long-term investment potential
            - Risk management considerations
            
            Format your response in clear sections and provide specific, actionable insights in Korean."""
        }]


        # 메시지 구성
        messages = [{
            'role': 'user',
            'content': [{
                'text': f'For {ticker}, I need: \
                    1. Current stock price and trading data \
                    2. Technical analysis of price movements \
                    3. Market sentiment analysis \
                    4. Trading recommendations'
                }]
        }]

        # 첫번째 Converse API 호출
        # 목적: 사용자의 초기 요청을 처리하고 필요한 도구(tool) 사용을 결정
        try:
            # 첫 번째 API 응답 처리
            response = client.converse(
                modelId='anthropic.claude-3-sonnet-20240229-v1:0',
                messages=messages,
                toolConfig=tool_config,
                inferenceConfig={
                    'temperature': 0.5,
                    'maxTokens': 1000
                }
            )

            # 응답 내용 확인 및 처리
            if 'output' in response and 'message' in response['output']:
                message_content = response['output']['message']['content']
                print("Initial Claude Response:")
                print(message_content)
                print("-" * 50)
                
                # Tool 사용이 필요한 경우 체크 (수정된 부분)
                if message_content:
                    has_tool_use = any('toolUse' in item for item in message_content)
                    if has_tool_use:
                        # Tool 사용이 있는 항목 찾기
                        tool_use = next(item['toolUse'] for item in message_content if 'toolUse' in item)
                        
                        # 주식 데이터 가져오기
                        stock_data = get_stock_data(ticker)
                        print("\nTool Use:")
                        print("Tool: get_stock_data")
                        print(f"Input: {{'ticker': '{ticker}'}}")
                        print("\n1. Stock Data Retrieved:")
                        print(stock_data)
                        print("-" * 50)

                        # 기술적 지표 데이터를 가져오기
                        if stock_data['status'] == 'success':
                            technical_analysis = analyze_technical_indicators(stock_data['data'])
                            print("\nTool Use:")
                            print("Tool: analyze_technical_indicators")
                            print(f"Input: {stock_data['data']}")
                            print("\n2. Technical Analysis:")
                            print(technical_analysis)
                            print("-" * 50)

                            # 뉴스 헤드라인 기반 시장 감성 분석
                            market_sentiment = get_market_sentiment(ticker)
                            print("\nTool Use:")
                            print("Tool: get_market_sentiment")
                            print(f"Input: {{'ticker': '{ticker}'}}")
                            print("\n3. Market Sentiment:")
                            print(market_sentiment)
                            print("-" * 50)

                        # 메시지 체인에 Tool 사용 결과 추가
                        messages.extend([
                            {
                                'role': 'assistant',
                                'content': [{'toolUse': tool_use}]
                            },
                            {
                                'role': 'user',
                                'content': [{
                                    'toolResult': {
                                        'toolUseId': tool_use['toolUseId'],
                                        'content': [{
                                            'json': {
                                                'stock_data': stock_data,
                                                'technical_analysis': technical_analysis,
                                                'market_sentiment': market_sentiment
                                            }
                                        }],
                                        'status': 'success'
                                    }
                                }]
                            }
                        ])
                        
                        print("\nGenerating final analysis...")
                        # 최종 응답 생성
                        final_response = client.converse(
                            modelId='anthropic.claude-3-sonnet-20240229-v1:0',
                            system=system_prompt,
                            messages=messages,
                            toolConfig=tool_config,
                            inferenceConfig={
                                "temperature": 0.7,
                                "maxTokens": 1000,
                                "stopSequences": []
                            }
                        )
                        
                        print("\nFinal Response Generated:")
                        print(final_response['output']['message']['content'][0]['text'])
                        return final_response['output']['message']['content'][0]['text']
        
                    else:
                        print("\nClaude did not request any tools.")
                        return message_content[0]['text']
    
            else:
                return "Error: Invalid response format"

        except Exception as e:
            return f"Error processing request: {str(e)}"
    except Exception as e:
        return f"Error in process_stock_request: {str(e)}"

In [9]:
ticker = input("Enter stock ticker (or 'quit' to exit): ").upper()
# ticker = 'AMZN'
result = process_stock_request(client, ticker)
print("\nResult:")
print(result)
print("-" * 50)

Initial Claude Response:
[{'text': 'Here are the steps to get the requested data and analysis for AMZN stock:'}, {'toolUse': {'toolUseId': 'tooluse_BP2TGgUuTvuBZKhEhuazZQ', 'name': 'get_stock_data', 'input': {'ticker': 'AMZN'}}}]
--------------------------------------------------

Tool Use:
Tool: get_stock_data
Input: {'ticker': 'AMZN'}

1. Stock Data Retrieved:
{'status': 'success', 'data': {'date': '2024-11-12', 'open': 208.37, 'high': 209.54, 'low': 206.01, 'volume': 38942918}}
--------------------------------------------------

Tool Use:
Tool: analyze_technical_indicators
Input: {'date': '2024-11-12', 'open': 208.37, 'high': 209.54, 'low': 206.01, 'volume': 38942918}

2. Technical Analysis:
{'daily_range': 3.530000000000001, 'volume_analysis': 'high'}
--------------------------------------------------

Tool Use:
Tool: get_market_sentiment
Input: {'ticker': 'AMZN'}

3. Market Sentiment:
{'sentiment_score': 0, 'recent_news': [{'title': 'Meet the Monster Stock that Continues to Crush 